In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
a = load_dataset(
    "/inspire/hdd/ws-8207e9e2-e733-4eec-a475-cfa1c36480ba/embodied-multimodality/public/zfhe/data/OpenR1-Math-220k-raw"
)["train"]
a_extended = load_dataset(
    "/inspire/hdd/ws-8207e9e2-e733-4eec-a475-cfa1c36480ba/embodied-multimodality/public/zfhe/data/OpenR1-Math-220k-raw/extended"
)["train"]
print(a)
a_extended

NameError: name 'load_dataset' is not defined

In [2]:
from datasets import concatenate_datasets, load_dataset

ds = load_dataset(
    "/inspire/hdd/ws-8207e9e2-e733-4eec-a475-cfa1c36480ba/embodied-multimodality/public/zfhe/data/OpenR1-Math-220k-raw"
)["train"]
ds_extended = load_dataset(
    "/inspire/hdd/ws-8207e9e2-e733-4eec-a475-cfa1c36480ba/embodied-multimodality/public/zfhe/data/OpenR1-Math-220k-raw/extended"
)["train"]
ds = concatenate_datasets([ds, ds_extended])
ds

Dataset({
    features: ['problem', 'solution', 'answer', 'problem_type', 'question_type', 'source', 'uuid', 'is_reasoning_complete', 'generations', 'correctness_math_verify', 'correctness_llama', 'finish_reasons', 'correctness_count', 'messages'],
    num_rows: 225129
})

In [6]:
from transformers import AutoTokenizer

models = {
    "b": "/inspire/hdd/ws-8207e9e2-e733-4eec-a475-cfa1c36480ba/embodied-multimodality/public/zfhe/jiaxing_projects/Language-Model-SAEs/exp/diff_models/Llama-3.1-8B",
    "i": "/inspire/hdd/ws-8207e9e2-e733-4eec-a475-cfa1c36480ba/embodied-multimodality/public/zfhe/jiaxing_projects/Language-Model-SAEs/exp/diff_models/Llama-3.1-8B-Instruct",
    "o": "/inspire/hdd/ws-8207e9e2-e733-4eec-a475-cfa1c36480ba/embodied-multimodality/public/zfhe/jiaxing_projects/Language-Model-SAEs/exp/diff_models/DeepSeek-R1-Distill-Llama-8B",
}

tokenizers = {k: AutoTokenizer.from_pretrained(v) for k, v in models.items()}
tokenizers

{'b': PreTrainedTokenizerFast(name_or_path='/inspire/hdd/ws-8207e9e2-e733-4eec-a475-cfa1c36480ba/embodied-multimodality/public/zfhe/jiaxing_projects/Language-Model-SAEs/exp/diff_models/Llama-3.1-8B', vocab_size=128000, model_max_length=131072, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|begin_of_text|>', 'eos_token': '<|end_of_text|>'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
 	128000: AddedToken("<|begin_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	128001: AddedToken("<|end_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	128002: AddedToken("<|reserved_special_token_0|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	128003: AddedToken("<|reserved_special_token_1|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	128004: AddedToken("<|finetune_right_pad_id|

In [16]:
tokenizers["b"] = AutoTokenizer.from_pretrained(
    "/inspire/hdd/ws-8207e9e2-e733-4eec-a475-cfa1c36480ba/embodied-multimodality/public/zfhe/jiaxing_projects/Language-Model-SAEs/exp/diff_models/Llama-3.1-8B"
)

In [19]:
tokenizers["o"] = AutoTokenizer.from_pretrained(
    "/inspire/hdd/ws-8207e9e2-e733-4eec-a475-cfa1c36480ba/embodied-multimodality/public/zfhe/jiaxing_projects/Language-Model-SAEs/exp/diff_models/DeepSeek-R1-Distill-Llama-8B"
)

In [3]:
ds[196]

{'problem': '\n3. Determine all pairs $m>n$ of positive integers such that\n\n$$\n1=\\operatorname{gcd}(n+1, m+1)=\\operatorname{gcd}(n+2, m+2)=\\cdots=\\operatorname{gcd}(m, 2 m-n)\n$$\n',
 'solution': '\nSolution: Observe that $1=\\operatorname{gcd}(n+r, m+r)=\\operatorname{gcd}(n+r, m-n)$. Thus each of the $m-n$ consecutive positive integers $n+1, n+2, \\ldots, m$ is coprime to $m-n$. Since one of these is necessarily a multiple of $m-n$, this is possible only when $m-n=1$. Hence each pair is of the form $(n, n+1)$, where $n \\in \\mathbb{N}$.\n',
 'answer': '(n,n+1)',
 'problem_type': 'Number Theory',
 'question_type': 'math-word-problem',
 'source': 'olympiads',
 'uuid': 'c1fd5aa6-dfca-58be-b8a3-17ab8a78bd64',
 'is_reasoning_complete': [True, True],
 'generations': ['<think>\nOkay, so I need to find all pairs of positive integers m and n where m is greater than n, and the greatest common divisors (gcds) from gcd(n+1, m+1) up to gcd(m, 2m - n) are all equal to 1. Hmm, that seems a 

In [4]:
def get_chat_template(example, tokenizer):
    messages = [{"role": m["from"], "content": m["value"]} for m in example["messages"]]

    return {
        "text": tokenizer.apply_chat_template(messages, tokenize=False)
        .removeprefix(tokenizer.bos_token)
        .removesuffix(tokenizer.eos_token)
    }

In [22]:
b = get_chat_template(ds[196], tokenizers["b"])
b

{'text': '<|q_start|>Q: <|q_end|>\n\n\n3. Determine all pairs $m>n$ of positive integers such that\n\n$$\n1=\\operatorname{gcd}(n+1, m+1)=\\operatorname{gcd}(n+2, m+2)=\\cdots=\\operatorname{gcd}(m, 2 m-n)\n$$\n<|a_start|>\nA: <|a_end|>\n\n<think>\nOkay, so I need to find all pairs of positive integers m and n where m is greater than n, and the greatest common divisors (gcds) from gcd(n+1, m+1) up to gcd(m, 2m - n) are all equal to 1. Hmm, that seems a bit complex, but let me break it down step by step.\n\nFirst, let\'s parse the problem statement again. We have pairs (m, n) with m > n, and for each k from 1 up to m - n, the gcd(n + k, m + k) should be 1. Wait, actually, the way it\'s written is from gcd(n+1, m+1) to gcd(m, 2m - n). Let me check how many terms there are here.\n\nStarting with gcd(n+1, m+1): the first term is when we add 1 to both n and m. Then the next is gcd(n+2, m+2), and so on, up to gcd(m, 2m - n). Let me see how many terms this is. The first part is n+1, and the l

In [10]:
i = get_chat_template(ds[196], tokenizers["i"])
i

{'text': '<|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n\n3. Determine all pairs $m>n$ of positive integers such that\n\n$$\n1=\\operatorname{gcd}(n+1, m+1)=\\operatorname{gcd}(n+2, m+2)=\\cdots=\\operatorname{gcd}(m, 2 m-n)\n$$\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n<think>\nOkay, so I need to find all pairs of positive integers m and n where m is greater than n, and the greatest common divisors (gcds) from gcd(n+1, m+1) up to gcd(m, 2m - n) are all equal to 1. Hmm, that seems a bit complex, but let me break it down step by step.\n\nFirst, let\'s parse the problem statement again. We have pairs (m, n) with m > n, and for each k from 1 up to m - n, the gcd(n + k, m + k) should be 1. Wait, actually, the way it\'s written is from gcd(n+1, m+1) to gcd(m, 2m - n). Let me check how many terms there are here.\n\nStarting with gcd(n+1, m+1): the first te

In [12]:
o = get_chat_template(ds[196], tokenizers["o"])
o

{'text': '<｜User｜>\n\n\n3. Determine all pairs $m>n$ of positive integers such that\n\n$$\n1=\\operatorname{gcd}(n+1, m+1)=\\operatorname{gcd}(n+2, m+2)=\\cdots=\\operatorname{gcd}(m, 2 m-n)\n$$\n<｜Assistant｜>\n\n<think>\nOkay, so I need to find all pairs of positive integers m and n where m is greater than n, and the greatest common divisors (gcds) from gcd(n+1, m+1) up to gcd(m, 2m - n) are all equal to 1. Hmm, that seems a bit complex, but let me break it down step by step.\n\nFirst, let\'s parse the problem statement again. We have pairs (m, n) with m > n, and for each k from 1 up to m - n, the gcd(n + k, m + k) should be 1. Wait, actually, the way it\'s written is from gcd(n+1, m+1) to gcd(m, 2m - n). Let me check how many terms there are here.\n\nStarting with gcd(n+1, m+1): the first term is when we add 1 to both n and m. Then the next is gcd(n+2, m+2), and so on, up to gcd(m, 2m - n). Let me see how many terms this is. The first part is n+1, and the last part is m. So the numbe

In [13]:
from functools import partial

# ds_b = ds.map(partial(get_chat_template, tokenizer=tokenizers['b']), remove_columns=ds.column_names, num_proc=32)
ds_i = ds.map(partial(get_chat_template, tokenizer=tokenizers["i"]), remove_columns=ds.column_names, num_proc=64)
# ds_o = ds.map(partial(get_chat_template, tokenizer=tokenizers['o']), remove_columns=ds.column_names, num_proc=32)
# ds_b.save_to_disk('/inspire/hdd/ws-8207e9e2-e733-4eec-a475-cfa1c36480ba/embodied-multimodality/public/zfhe/jiaxing_projects/Language-Model-SAEs/exp/diff_datas/OpenR1-Math-220k-b')
ds_i.save_to_disk(
    "/inspire/hdd/ws-8207e9e2-e733-4eec-a475-cfa1c36480ba/embodied-multimodality/public/zfhe/jiaxing_projects/Language-Model-SAEs/exp/diff_datas/OpenR1-Math-220k-i"
)
# ds_o.save_to_disk('/inspire/hdd/ws-8207e9e2-e733-4eec-a475-cfa1c36480ba/embodied-multimodality/public/zfhe/jiaxing_projects/Language-Model-SAEs/exp/diff_datas/OpenR1-Math-220k-o')

Map (num_proc=64):   0%|          | 0/225129 [00:00<?, ? examples/s]

Saving the dataset (0/7 shards):   0%|          | 0/225129 [00:00<?, ? examples/s]

In [1]:
from datasets import load_from_disk

a = load_from_disk(
    "/inspire/hdd/ws-8207e9e2-e733-4eec-a475-cfa1c36480ba/embodied-multimodality/public/zfhe/jiaxing_projects/Language-Model-SAEs/exp/diff_datas/OpenR1-Math-220k-i"
)["text"]
a[196]

In [54]:
ds_o.save_to_disk(
    "/inspire/hdd/ws-8207e9e2-e733-4eec-a475-cfa1c36480ba/embodied-multimodality/public/zfhe/jiaxing_projects/Language-Model-SAEs/exp/diff_datas/OpenR1-Math-220k-o"
)

Saving the dataset (0/7 shards):   0%|          | 0/225129 [00:00<?, ? examples/s]

In [30]:
def construct_analyze_data(example):
    m = example["messages"]
    return {"text": ("\n\n" + m[0]["value"] + "\n\n" + m[1]["value"]).replace("<think>", "").replace("</think>", "")}

In [32]:
b1 = construct_analyze_data(ds[0])
print(b1["text"])
# ds[196]['messages'][0]['value']



## Task B-1.3.

A ship traveling along a river has covered $24 \mathrm{~km}$ upstream and $28 \mathrm{~km}$ downstream. For this journey, it took half an hour less than for traveling $30 \mathrm{~km}$ upstream and $21 \mathrm{~km}$ downstream, or half an hour more than for traveling $15 \mathrm{~km}$ upstream and $42 \mathrm{~km}$ downstream, assuming that both the ship and the river move uniformly.

Determine the speed of the ship in still water and the speed of the river.


Okay, so I need to find the speed of the ship in still water and the speed of the river. Let me start by recalling that when a ship is moving upstream, its effective speed is the speed of the ship minus the speed of the river. Conversely, when moving downstream, its effective speed is the ship's speed plus the river's speed. 

Let me denote the speed of the ship in still water as \( v \) (in km/h) and the speed of the river as \( r \) (also in km/h). Then, the upstream speed would be \( v - r \), and the downstr

In [33]:
ds_b = ds.map(construct_analyze_data, remove_columns=ds.column_names, num_proc=32)
ds_b.save_to_disk(
    "/inspire/hdd/ws-8207e9e2-e733-4eec-a475-cfa1c36480ba/embodied-multimodality/public/zfhe/jiaxing_projects/Language-Model-SAEs/exp/diff_datas/analyze_datas/OpenR1-Math-220k-b"
)

Map (num_proc=32):   0%|          | 0/225129 [00:00<?, ? examples/s]

Saving the dataset (0/7 shards):   0%|          | 0/225129 [00:00<?, ? examples/s]

In [15]:
x = load_from_disk(
    "/inspire/hdd/ws-8207e9e2-e733-4eec-a475-cfa1c36480ba/embodied-multimodality/public/zfhe/jiaxing_projects/Language-Model-SAEs/exp/diff_datas/analyze_datas/OpenR1-Math-220k"
)["text"]
print(x[2])



Prove that number $1$ can be represented as a sum of a finite number $n$ of real numbers, less than $1,$ not necessarily  distinct, which contain in their decimal representation only the digits $0$ and/or $7.$ Which is the least possible number $n$?


Okay, so the problem is to prove that the number 1 can be represented as a sum of a finite number n of real numbers, each less than 1, where each number in the sum has a decimal representation consisting only of the digits 0 and/or 7. And then we need to find the least possible n.

First, let me make sure I understand the problem correctly. We need to find the smallest n such that there exist numbers a_1, a_2, ..., a_n, each less than 1, and each a_i has a decimal expansion with only 0s and 7s. When we add these numbers together, the sum is exactly 1.

Hmm. So each of these numbers is like 0.7, 0.07, 0.777, 0.0707, etc. But each must be less than 1, so none of them can be 0.777... (with infinitely many 7s) because that's equal to 7/9, w

In [48]:
import safetensors.torch as safe
from transformers import AutoTokenizer

l = safe.load_file
tokenizer_b = AutoTokenizer.from_pretrained(
    "/inspire/hdd/ws-8207e9e2-e733-4eec-a475-cfa1c36480ba/embodied-multimodality/public/zfhe/jiaxing_projects/Language-Model-SAEs/exp/diff_models/Llama-3.1-8B"
)
tokenizer_i = AutoTokenizer.from_pretrained(
    "/inspire/hdd/ws-8207e9e2-e733-4eec-a475-cfa1c36480ba/embodied-multimodality/public/zfhe/jiaxing_projects/Language-Model-SAEs/exp/diff_models/Llama-3.1-8B-Instruct"
)
tokenizer_o = AutoTokenizer.from_pretrained(
    "/inspire/hdd/ws-8207e9e2-e733-4eec-a475-cfa1c36480ba/embodied-multimodality/public/zfhe/jiaxing_projects/Language-Model-SAEs/exp/diff_models/DeepSeek-R1-Distill-Llama-8B"
)

In [86]:
def remove_all_suffixes(s, suffix):
    while s.endswith(suffix):
        s = s[: -len(suffix)]
    return s


# brf = l('/inspire/hdd/global_user/hezhengfu-240208120186/jiaxing_activations/reasondata-b-2d-801M-f/blocks.15.hook_resid_post/shard-0-chunk-00000004.safetensors')
# i = remove_all_suffixes(tokenizer_b.decode(brf['tokens'][0].tolist()),'<|end_of_text|>')
# # for c in range(len(i)):
# #     if i[:c] != x[2][:c]:
# #         print(c)
# #         break
# i in x[4]

# 666 have a "one char but multiple tokens"
for i in range(667, 1000):
    brf = l(
        f"/inspire/hdd/global_user/hezhengfu-240208120186/jiaxing_activations/reasondata-b-2d-801M-f/blocks.15.hook_resid_post/shard-0-chunk-{i:08d}.safetensors"
    )
    b = remove_all_suffixes(tokenizer_b.decode(brf["tokens"][0].tolist()), "<|end_of_text|>")
    # print(b in x[i])
    if b not in x[i]:
        print(i)
        break

In [80]:
brf = l(
    "/inspire/hdd/global_user/hezhengfu-240208120186/jiaxing_activations/reasondata-b-2d-801M-f/blocks.15.hook_resid_post/shard-0-chunk-00000666.safetensors"
)
b = remove_all_suffixes(tokenizer_b.decode(brf["tokens"][0].tolist()), "<|end_of_text|>")
for c in range(len(b)):
    if b[:c] != x[666][:c]:
        print(c)
        break
# print(b)
li = brf["tokens"][0].tolist()
tokenizer_o.decode(60094)

'�'

In [85]:
tokenizer_b.encode("≈")

[60094, 230]

In [57]:
print(x[666])



7. The function
$$
f(x)=\frac{\sin x-1}{\sqrt{3-2 \cos x-2 \sin x}}(0 \leqslant x \leqslant 2 \pi)
$$

has the range . $\qquad$


Okay, let's try to figure out the range of this function. The function is f(x) = (sin x - 1) divided by the square root of (3 - 2 cos x - 2 sin x), and x is between 0 and 2π. Hmm, so I need to find all possible values that f(x) can take. 

First, let me understand the components of the function. The numerator is sin x - 1. Since sin x ranges between -1 and 1, sin x - 1 will range between -2 and 0. So the numerator is always non-positive, right? Because the maximum value of sin x is 1, so sin x -1 is at most 0, and the minimum is -2. 

Then the denominator is sqrt(3 - 2 cos x - 2 sin x). Since it's a square root, the expression inside must be non-negative. So 3 - 2 cos x - 2 sin x ≥ 0. Let me check if that's always true or if there are restrictions. Maybe we can simplify the expression inside the square root. 

Let me denote the denominator's expression as 

In [23]:
print(x[2][:671])
print(i[:671])



Prove that number $1$ can be represented as a sum of a finite number $n$ of real numbers, less than $1,$ not necessarily  distinct, which contain in their decimal representation only the digits $0$ and/or $7.$ Which is the least possible number $n$?


Okay, so the problem is to prove that the number 1 can be represented as a sum of a finite number n of real numbers, each less than 1, where each number in the sum has a decimal representation consisting only of the digits 0 and/or 7. And then we need to find the least possible n.

First, let me make sure I understand the problem correctly. We need to find the smallest n such that there exist numbers a_1, a_2, ...


Prove that number $1$ can be represented as a sum of a finite number $n$ of real numbers, less than $1,$ not necessarily  distinct, which contain in their decimal representation only the digits $0$ and/or $7.$ Which is the least possible number $n$?


Okay, so the problem is to prove that the number 1 can be represented as a

In [45]:
orf = l(
    "/inspire/hdd/global_user/hezhengfu-240208120186/jiaxing_activations/reasondata-o-2d-801M-f/blocks.15.hook_resid_post/shard-0-chunk-00000002.safetensors"
)
o = remove_all_suffixes(tokenizer_o.decode(orf["tokens"][0].tolist()[:164]), "<|end_of_text|>")
print(o)
# print(o[:671])
print(tokenizer_o.decode(orf["tokens"][0].tolist()[163]))
print(tokenizer_b.decode(brf["tokens"][0].tolist()[163]))
print(tokenizer_o.decode(61455))
print(tokenizer_o.decode(61453))
print(tokenizer_b.decode(61455))
print(tokenizer_b.decode(61453, clean_up_tokenization_spaces=False))
print(orf["tokens"][0].tolist()[163])
print(brf["tokens"][0].tolist()[163])



Prove that number $1$ can be represented as a sum of a finite number $n$ of real numbers, less than $1,$ not necessarily  distinct, which contain in their decimal representation only the digits $0$ and/or $7.$ Which is the least possible number $n$?


Okay, so the problem is to prove that the number 1 can be represented as a sum of a finite number n of real numbers, each less than 1, where each number in the sum has a decimal representation consisting only of the digits 0 and/or 7. And then we need to find the least possible n.

First, let me make sure I understand the problem correctly. We need to find the smallest n such that there exist numbers a_1, a_2, ...,
 ...,
...,
 DOC
 ...,
 DOC
 ...,
61453
61453
